In [1]:
import torch
import pickle
import numpy as np
from argparse import Namespace
from torch.utils.data import DataLoader
import torch.nn.functional as F
from heterogt.utils.tokenizer import EHRTokenizer
from heterogt.utils.dataset import FineTuneEHRDataset, batcher, expand_level3
from heterogt.utils.train import train_with_early_stopping, PHENO_ORDER
from heterogt.utils.seed import set_random_seed
from heterogt.model.model import HeteroGTFineTune

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.13.1
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
set_random_seed(123)

[INFO] Random seed set to 123


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
config = Namespace(
    dataset = "MIMIC-IV",
    tasks = ["death", "readmission", "stay", "next_diag_6m", "next_diag_12m"],
    task_index = 4,  # index of the task to train
    token_type = ["diag", "med", "lab", "pro"],
    special_tokens = ["[PAD]", "[CLS]"],
    # only for tf layer
    attn_mask_dicts = [{1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 5:[6,7], 6:[1,2,3,4,5,6,7], 7:[1,2,3,4,5,6,7]}, 
                       {1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 5:[6,7], 6:[1,2,3,4,5,6,7], 7:[1,2,3,4,5,6,7]}], 
    d_model = 64,
    num_heads = 4,
    batch_size = 32,
    lr = 1e-3,
    epochs = 500,
    early_stop_patience = 5,
    group_code_thre = 5,  # if there are group_code_thre diag codes belongs to the same group ICD code, then the group code is generated
    use_pretrained_model = True,
    pretrain_mask_rate = 0.7,
    pretrain_cls_ontology_weight = 5e-2,
    pretrain_visit_ontology_weight = 5e-2,
    pretrain_adm_type_weight = 0,
    dec_loss_lambda = 1e-2,
)

In [5]:
full_data_path = f"./data_process/{config.dataset}-processed/mimic.pkl"  # for tokenizer
curr_task = config.tasks[config.task_index]
print("Current task:", curr_task)
if curr_task == "next_diag_6m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_6m.pkl"
elif curr_task == "next_diag_12m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_12m.pkl"
else:
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_downstream.pkl"

Current task: next_diag_12m


In [6]:
ehr_full_data = pickle.load(open(full_data_path, 'rb'))
group_code_sentences = [expand_level3()[1]]
diag_sentences = ehr_full_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_full_data["NDC"].values.tolist()
lab_sentences = ehr_full_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_full_data["PRO_CODE"].values.tolist()
age_sentences = [[str(c)] for c in set(ehr_full_data["AGE"].values.tolist())] # important of [[]]
adm_type_sentences = ehr_full_data["ADMISSION_TYPE"].values.tolist()
max_admissions = ehr_full_data.groupby("SUBJECT_ID")["HADM_ID"].nunique().max()
config.max_num_adms = max_admissions
print(f"Max admissions per patient: {config.max_num_adms}")

Max admissions per patient: 8


In [7]:
tokenizer = EHRTokenizer(age_sentences, group_code_sentences, diag_sentences, med_sentences, lab_sentences, 
                         pro_sentences, special_tokens=config.special_tokens, adm_types_sentences=adm_type_sentences)
config.label_vocab_size = len(PHENO_ORDER)  # a set of predifined diseases
config.global_vocab_size = len(tokenizer.vocab.id2word)
config.age_vocab_size = tokenizer.token_number("age")
config.group_code_vocab_size = tokenizer.token_number("group")
print(f"Age vocabulary size: {config.age_vocab_size}")
print(f"Group code vocabulary size: {config.group_code_vocab_size}")

Age vocabulary size: 20
Group code vocabulary size: 19


In [8]:
train_data, val_data, test_data = pickle.load(open(finetune_data_path, 'rb'))
# example label percentage
print("Percentage of DEATH in test dataset:",
      (test_data["DEATH"] == True).mean() * 100, "%")

print("Percentage of READMISSION in test dataset:",
      (test_data["READMISSION"] == 1).mean() * 100, "%")

print("Percentage of STAY>7 days in test dataset:",
      (test_data["STAY_DAYS"] > 7).mean() * 100, "%")

Percentage of DEATH in test dataset: 3.327211839688884 %
Percentage of READMISSION in test dataset: 79.05368910014043 %
Percentage of STAY>7 days in test dataset: 34.87090850167441 %


In [9]:
train_dataset = FineTuneEHRDataset(train_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
val_dataset = FineTuneEHRDataset(val_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                 max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
test_dataset = FineTuneEHRDataset(test_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)

In [10]:
long_adm_seq_crite = 3
val_long_seq_idx, test_long_seq_idx = [], []
for i in range(len(val_dataset)):
    hadm_id = list(val_dataset.records.keys())[i]
    num_adms = len(val_dataset.records[hadm_id])
    if num_adms >= long_adm_seq_crite:
        val_long_seq_idx.append(i)
for i in range(len(test_dataset)):
    hadm_id = list(test_dataset.records.keys())[i]
    num_adms = len(test_dataset.records[hadm_id])
    if num_adms >= long_adm_seq_crite:
        test_long_seq_idx.append(i)
print(len(val_long_seq_idx), len(test_long_seq_idx))

857 876


In [11]:
num_group_code = []
for i in range(len(train_dataset)):
    input_ids, token_types, adm_index, age_ids, diag_group_codes, labels = train_dataset[i]
    count = (token_types[0] == 6).sum().item()
    num_group_code.append(count)
print("Mean group token numer per patient", np.mean(num_group_code))

Mean group token numer per patient 0.874014598540146


In [12]:
train_dataloader = DataLoader(
    train_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=True,
    batch_size=config.batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

In [13]:
for batch in train_dataloader:
    pass  # just to check if the dataloader works
for batch in val_dataloader:
    pass  # just to check if the dataloader works
for batch in test_dataloader:
    pass  # just to check if the dataloader works
print("All pass!")

All pass!


In [14]:
if curr_task in ["death", "stay", "readmission"]:
    eval_metric = "f1"
    task_type = "binary"
    loss_fn = F.binary_cross_entropy_with_logits
else:
    eval_metric = "f1"
    task_type = "l2r"
    loss_fn = lambda x, y: F.binary_cross_entropy_with_logits(x, y)

In [15]:
input_ids, token_types, adm_index, age_ids, diag_code_group_dicts, labels = next(iter(train_dataloader))
print("Input IDs shape:", input_ids.shape)
print("Token Types shape:", token_types.shape)
print("Admission Index shape:", adm_index.shape)
print("Age IDs shape:", age_ids.shape)
print("Diag Code Group Dict number:", len(diag_code_group_dicts))
print("Labels shape:", labels.shape)

Input IDs shape: torch.Size([32, 233])
Token Types shape: torch.Size([32, 233])
Admission Index shape: torch.Size([32, 233])
Age IDs shape: torch.Size([32, 6])
Diag Code Group Dict number: 32
Labels shape: torch.Size([32, 18])


# Model Walkthrough

In [16]:
# load pretrained model
if config.use_pretrained_model:
    pretrain_exp_name = (
    f"{config.dataset}-{config.pretrain_mask_rate}-{config.d_model}-{config.pretrain_cls_ontology_weight}-{config.pretrain_visit_ontology_weight}-{config.pretrain_adm_type_weight}"
)
    print(pretrain_exp_name)
    save_path = "./pretrained_models/" + pretrain_exp_name
    state_dict = torch.load(f"{save_path}/pretrained_model.pt", map_location="cpu")

MIMIC-IV-0.7-64-0.05-0.05-0


In [ ]:
final_metrics, final_long_seq_metrics = [], []
for i in range(15):
    print(f"================================{i+1}==================================")
    model = HeteroGTFineTune(tokenizer=tokenizer, token_types=config.token_type, d_model=config.d_model, num_heads=config.num_heads, layer_types=['gnn', 'tf', 'gnn', 'tf'], max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, label_vocab_size=config.label_vocab_size, attn_mask_dicts=config.attn_mask_dicts,
                     use_cls_cat=True).to(device)
    if config.use_pretrained_model:
        model.load_weight(state_dict)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric, best_test_long_seq_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                                                            optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                                                            dec_loss_lambda=config.dec_loss_lambda, 
                                                                            val_long_seq_idx=val_long_seq_idx, test_long_seq_idx=test_long_seq_idx, 
                                                                            eval_metric=eval_metric, return_model=False)
    final_metrics.append(best_test_metric)
    final_long_seq_metrics.append(best_test_long_seq_metric)

================================1==================================


Running inference: 100%|██████████| 158/158 [00:07<00:00, 21.25it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          55.3191   2.2472   4.3189   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         80.4154  19.7955  31.7702   
Chronic kidney disease                       86.1340  63.0267  72.7905   
Chronic obstructive pulmonary disease         0.0000   0.0000   0.0000   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    86.1566  41.7845  56.2760   
Coronary atherosclerosis and related         79.5389  68.0362  73.3392   
Disorders of lipid metabolism                69.5375  69.1332  69.3347   
Essential hypertension                       73.2880  82.0867  77.4382   
Fluid and electrolyte disorders              64.9123   2.2811   4.4074   
Gastrointestinal hemorrhage           

Running inference: 100%|██████████| 159/159 [00:07<00:00, 21.22it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          46.3415   1.6393   3.1667   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         77.0642  18.9902  30.4716   
Chronic kidney disease                       89.0608  63.6148  74.2173   
Chronic obstructive pulmonary disease         0.0000   0.0000   0.0000   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    85.6089  40.1037  54.6204   
Coronary atherosclerosis and related         78.4332  68.1345  72.9220   
Disorders of lipid metabolism                67.4169  68.5559  67.9816   
Essential hypertension                       73.1126  82.5523  77.5462   
Fluid and electrolyte disorders              54.2857   2.3945   4.5866   
Gastrointestinal hemorrhage           

...

Running inference: 100%|██████████| 158/158 [00:07<00:00, 21.34it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          53.7190  11.2360  18.5847   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         67.1071  70.4894  68.7567   
Chronic kidney disease                       82.9534  71.9690  77.0718   
Chronic obstructive pulmonary disease        60.0000   8.0357  14.1732   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    72.6997  63.5159  67.7982   
Coronary atherosclerosis and related         80.7725  77.3213  79.0092   
Disorders of lipid metabolism                71.7937  86.0994  78.2985   
Essential hypertension                       78.1809  74.9404  76.5264   
Fluid and electrolyte disorders              55.5000  20.5302  29.9730   
Gastrointestinal hemorrhage           

Running inference: 100%|██████████| 159/159 [00:07<00:00, 21.25it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          53.7190  11.2166  18.5582   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         64.0971  67.6714  65.8358   
Chronic kidney disease                       87.5357  72.6125  79.3788   
Chronic obstructive pulmonary disease        54.0000   6.3380  11.3445   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    74.2769  62.1435  67.6706   
Coronary atherosclerosis and related         79.3522  78.4628  78.9050   
Disorders of lipid metabolism                69.8558  87.1390  77.5461   
Essential hypertension                       78.8021  75.4237  77.0759   
Fluid and electrolyte disorders              52.6154  21.5501  30.5767   
Gastrointestinal hemorrhage           

Running inference:  15%|█▌        | 24/158 [00:01<00:06, 21.16it/s]

In [ ]:
def topk_avg_performance_formatted(performances, long_seq_performances, k=5):
    metrics = ["f1", "auc", "prauc"]
    scores = {m: np.array([p[m] for p in performances]) for m in metrics}

    # 计算排名（值越大排名越靠前）
    ranks = {m: (-scores[m]).argsort().argsort() + 1 for m in metrics}
    avg_ranks = np.mean(np.stack([ranks[m] for m in metrics], axis=1), axis=1)

    # 选 top-k
    topk_idx = np.argsort(avg_ranks)[:k]
    final_avg = {m: np.mean([performances[i][m] for i in topk_idx]) for m in performances[0].keys()}
    final_std = {m: np.std([performances[i][m] for i in topk_idx], ddof=0) for m in performances[0].keys()}
    final_long_seq_avg = {m: np.mean([long_seq_performances[i][m] for i in topk_idx]) for m in long_seq_performances[0].keys()}
    final_long_seq_std = {m: np.std([long_seq_performances[i][m] for i in topk_idx], ddof=0) for m in long_seq_performances[0].keys()}

    # 打印结果（转百分比，均保留两位小数）
    print("Final Metrics:")
    for m in performances[0].keys():
        mean_val = final_avg[m]
        std_val = final_std[m]
        print(f"{m}: {mean_val:.2f} ± {std_val:.2f}")
    print("\nFinal Long Sequence Metrics:")
    for m in long_seq_performances[0].keys():
        mean_val = final_long_seq_avg[m]
        std_val = final_long_seq_std[m]
        print(f"{m}: {mean_val:.2f} ± {std_val:.2f}")

In [ ]:
topk_avg_performance_formatted(final_metrics, final_long_seq_metrics)